# Preparing an AF state in the Ising model

This notebook illustrates how to use Pulser to build a sequence for preparing an AF state in the Ising model. It is based on [10.1103/PhysRevX.8.021070](10.1103/PhysRevX.8.021070).

We begin by importing some basic modules:

In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import qutip

from pulser import Pulse, Sequence, Register
from pulser.waveforms import ConstantWaveform, RampWaveform
from pulser.devices import Chadoq2, MockDevice
from pulser.simulation import Simulation

# 1. 2D Square Array

## Waveforms 

We are realizing the following program

<img src="files/AF_Ising_program.png" alt="AF Pulse Sequence" style="width: 320px;"/>

This pulse is defined by the following parameters:

In [ ]:
# Parameters in MHz and ns
delta_0 = -30 * 2*np.pi
delta_f = 21 * 2*np.pi
Omega_max = 18 * 2*np.pi 

t_rise = 300
t_fall = 400
t_sweep = 1000 - (t_rise + t_fall)
#t_sweep = (delta_f - delta_0)/(2*np.pi*10) * 1000

We define our register with an interatomic distance equal to the Rydberg radius. This in turn depends on the maximum Rabi frequency in our pulse sequence.

In [ ]:
R_blockade = np.round((5.008e6/Omega_max)**(1/6), 3)

N_side = 3
reg = Register.rectangle(N_side, N_side, R_blockade, prefix='q')
print(f'Blockade Radius is: {R_blockade}µm.')
reg.draw()

## Creating my sequence

We compose our pulse with the following objects from Pulser:

In [ ]:
rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0., Omega_max), delta_0, 0.)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.), delta_f, 0.)

In [ ]:
seq = Sequence(reg, MockDevice)
seq.declare_channel('ising', 'rydberg_global')

seq.add(rise, 'ising')
seq.add(sweep, 'ising')
seq.add(fall, 'ising')

#print(seq)
seq.draw()

## Phase Diagram

The pulse sequence travels though the following path in the phase diagram of the system (the shaded area represents the antiferromagnetic phase):

In [ ]:
U = 1 * 2*np.pi
delta = []
omega = []
for x in seq._schedule['ising']:
    if isinstance(x.type,Pulse):
        omega += list(x.type.amplitude.samples/U)
        delta += list(x.type.detuning.samples/U)
        
fig, ax = plt.subplots()
ax.grid(True, which='both')

ax.set_ylabel(r"$\hbar\delta(t)/U$", fontsize=16)
ax.set_xlabel(r"$\hbar\Omega(t)/U$", fontsize=16)
ax.set_xlim(0,5)
#ax.set_ylim(-7,7)
ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')

y = np.arange(0.0, 6, 0.01)
x = 1.522*(1-0.25*(y-2)**2)
ax.fill_between(x, y, alpha=0.4)

ax.plot(omega,delta,'red',lw=2)
plt.show()


# Simulation

We now run a simulation of the sequence:

In [ ]:
simul = Simulation(seq)

The observables to measure will be the occupation operator $|r\rangle \langle r|_i$ on each site $i$ of the register, where the Rydberg state $|r\rangle$ represents the excited state.

In [ ]:
def occupation(j,N):
    up = qutip.basis(2,0)
    prod = [qutip.qeye(2) for _ in range(N)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)

In [ ]:
occup_list = [occupation(j,N_side*N_side) for j in range(N_side*N_side)]

We now run the simulation and plot the evolution of the expectation values, as well as a representation of the final values after the pulse is applied:

In [ ]:
simul.run(obs_list=occup_list, progress_bar=True)

for expv in simul.output.expect:
    plt.plot(expv)
    
res=np.zeros((N_side,N_side))
for i,ev in enumerate(simul.output.expect):
    res[i//N_side,i%N_side] = ev[-1]
plt.matshow(res, cmap='hot')
print(res)

## Spin-Spin Correlation Function

In [ ]:
simul.run(progress_bar=True)

Define a function that returns a list of all pairs $(i,j)$ whose distance is ${\bf r}_i - {\bf r}_j = (k R_b,l R_b)$ in the atomic array coordinate (both $k$ and $l$ are positive or negative integers within the size of the array):

In [ ]:
def get_corr_pairs(k, l, register):
    corr_pairs = []
    for i, qi in enumerate(register.qubits):
        for j, qj in enumerate(register.qubits):
            r_ij = np.round(register.qubits[qi]-register.qubits[qj],3)
            distance = np.linalg.norm(r_ij - R_blockade*np.array([k,l]))
            if distance<1:
                corr_pairs.append([i,j])
    return corr_pairs

In [ ]:
def get_corr_function(k, l, reg, state):
    N_qubits = len(reg.qubits)
    corr_pairs = get_corr_pairs(k, l, reg)
    
    operators = [occupation(j, N_qubits) for j in range(N_qubits)]
    covariance = 0
    for qi, qj in corr_pairs:
        covariance += qutip.expect(operators[qi]*operators[qj], state)
        covariance -= qutip.expect(operators[qi], state)*qutip.expect(operators[qj], state)
    return covariance/len(corr_pairs)
        
def get_full_corr_function(reg, state, config='square'):
    N_qubits = len(reg.qubits)
    
    correlation_function = {}
    if config == 'square':
        N_side = int(np.sqrt(N_qubits))
        for k in range(-N_side+1,N_side):
            for l in range(-N_side+1, N_side):
                correlation_function[(k,l)] = get_corr_function(k, l, reg, state)
    elif config == 'line':
        for k in range(-N_qubits+1,N_qubits):
            correlation_function[k] = get_corr_function(k, 0, reg, state)
    return correlation_function

In [ ]:
final = simul.output.states[-1]
correlation_function = get_full_corr_function(reg, final)

In [ ]:
expected_corr_function = {}
xi = 1.53
for k in range(-N_side+1,N_side):
    for l in range(-N_side+1,N_side):
        kk = np.abs(k)
        ll = np.abs(l)
        expected_corr_function[(k,l)] = (-1)**(kk + ll) * np.exp(-(kk + ll)/xi)

In [ ]:
A = 4*np.reshape(list(correlation_function.values()), (2*N_side-1,2*N_side-1))
B = A[2,2]*np.reshape(list(expected_corr_function.values()), (2*N_side-1,2*N_side-1))

fig, axes = plt.subplots(nrows=2, ncols=1)
im_A = axes[0].imshow(A, cmap='coolwarm')
axes[0].set_title(r'$g^{(2)}(k,l)$ after simulation')

im_B = axes[1].imshow(B, cmap='coolwarm')
#axes[1].set_title(r'Exponential $g^{(2)}(k,l)$ expected')

fig.colorbar(im_A, ax=axes.ravel().tolist())
plt.show()


In [ ]:
np.around(A,5)

In [ ]:
np.around(B,5)

## Néel Structure Factor

In [ ]:
def get_neel_structure_factor(reg, state):
    st_fac = 0
    for k in range(-N_side+1, N_side):
        for l in range(-N_side+1, N_side):
            kk = np.abs(k)
            ll = np.abs(l)
            if not (k == 0 and l == 0):
                st_fac += 4 * (-1)**(kk + ll) * get_corr_function(k,l,reg,state)
    return st_fac 

In [ ]:
def calculate_neel(det, omax=None):
    # Parameters in MHz and ns 
    U = 1. * 2*np.pi
    if omax:
        Omega_max=omax*U
    else:
        Omega_max = 18 * U
    delta_0 = -30 * 2*np.pi
    delta_f = det * 2*np.pi

    R_blockade = (5.008e6/Omega_max)**(1/6)

    N = 3
    reg = Register.rectangle(N, N, R_blockade, prefix='q')
    #print(f'Blockade Radius is: {R_blockade}µm.')
    #reg.draw()


    t_rise = 300
    t_fall = 400
    t_sweep = 1000 - (t_rise + t_fall)
    #t_sweep = int((delta_f - delta_0)/(2*np.pi*10) * 1000)

    rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0., Omega_max), delta_0, 0.)
    sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.)
    fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.), delta_f, 0.)

    seq = Sequence(reg, MockDevice)
    seq.declare_channel('ising', 'rydberg_global')

    seq.add(rise, 'ising')
    seq.add(sweep, 'ising')
    seq.add(fall, 'ising')

    #print(seq)
    #seq.draw()

    simul = Simulation(seq)
    simul.run()
    
    final = simul.output.states[-1]
    return get_neel_structure_factor(reg,final)

                                                                                                                                                                      

In [ ]:
detunings = np.linspace(-5,10,20)
results=[]
for det in detunings:
    results.append(calculate_neel(det))
plt.xlabel(r'$\hbar\delta_{final}/U$')
plt.ylabel(r'Néel Structure Factor $S_{Neel}$')
plt.plot(detunings, results, 'o', ls='solid')
plt.show()

# 2. 1D open Chain

One can also try other geometries. In a 1D open boundary chain, for example:

In [ ]:
# Parameters in MHz and ns
U = 1. * 2*np.pi 
delta_0 = -30 * 2*np.pi
delta_f = 21 * 2*np.pi
Omega_max = 18 * 2*np.pi
t_rise = 300
t_fall = 400
t_sweep = 1000 - (t_rise+t_fall)
#t_sweep = (delta_f - delta_0)/(2*np.pi*10) * 1000

# Register
R_blockade = np.round((5.008e6/Omega_max)**(1/6),3)
N_qubits = 13
reg = Register.rectangle(1,N_qubits,R_blockade, prefix='q')

reg.draw()


rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0., Omega_max), delta_0, 0.)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.), delta_f, 0.)

seq = Sequence(reg, MockDevice)
seq.declare_channel('ising', 'rydberg_global')

seq.add(rise, 'ising')
seq.add(sweep, 'ising')
seq.add(fall, 'ising')

#print(seq)
seq.draw()

The phase diagram is plotted below:

In [ ]:
#phase = {'omega':[], 'delta':[], 'time': range(max(seq._last(ch).tf for ch in seq.declared_channels))}
delta = []
omega = []
for x in seq._schedule['ising']:
    if isinstance(x.type,Pulse):
        omega += list(x.type.amplitude.samples/U)
        delta += list(x.type.detuning.samples/U)
        
fig, ax = plt.subplots()
ax.grid(True, which='both')

ax.set_ylabel(r"$\delta(t)/\hbar\,U$")
ax.set_xlabel(r"$\Omega(t)/\hbar\,U$")
ax.set_xlim(0,4)
ax.set_ylim(-7,3)
ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')

y = np.arange(0.0, 6, 0.01)
x = 0.5*(1-(y-1)**2)
ax.fill_between(x, y, color='green',alpha=0.4)

ax.plot(omega,delta,'red',lw=2)

In [ ]:
simul = Simulation(seq)
    
occup_list = [occupation(j, N_qubits) for j in range(N_qubits)]

simul.run(obs_list=occup_list, progress_bar=True)

The results are as follows:

In [ ]:
for expv in simul.output.expect:
    plt.plot(expv)
plt.show()
res =np.zeros((1,N_qubits))
for i,ev in enumerate(simul.output.expect):
    res[0,i] = ev[-1]
plt.matshow(res, cmap = 'hot')
print(res)

The results for the Spin-Spin correlation function are:

In [ ]:
simul.run(progress_bar=True)

In [ ]:
final = simul.output.states[-1]
correlation_function = get_full_corr_function(reg, final, config='line')

In [ ]:
expected_corr_function = {}
xi = 1.53
for k in range(-N_qubits+1,N_qubits):
    kk = np.abs(k)
    expected_corr_function[k] = (-1)**(kk) * np.exp(-kk/xi)

In [ ]:
A = 4*np.reshape(list(correlation_function.values()), (1,2*N_qubits-1))
B = A[0][10]*np.reshape(list(expected_corr_function.values()), (1,2*N_qubits-1))

print(A)
fig, axes = plt.subplots(nrows=2, ncols=1)
im_A = axes[0].imshow(A, cmap='coolwarm')
axes[0].set_title(r'$g^{(2)}(k,l)$ after simulation')

im_B = axes[1].imshow(B, cmap='coolwarm')
axes[1].set_title(r'Exponential $g^{(2)}(k,l)$ expected')

fig.colorbar(im_A, ax=axes.ravel().tolist())
plt.show()



In [ ]:
plt.plot(A[0],'o')
plt.plot(B[0],'o')